In [ ]:
import numpy as np
import glob
from PIL import Image
import json
import os
import shutil

# STUFF

In [ ]:
masks = glob.glob('/root/data/aquabyte-images/erko/raw/*/*.semantic.jpg') + glob.glob('/root/data/aquabyte-images/erko/raw/*/*.semantic.png')
print(len(masks))

In [ ]:
good_images = '/root/data/aquabyte-images/good_images2.json'

In [ ]:
test = json.load(open(good_images))

In [ ]:
for t in test:
    shutil.copy(t, os.path.join('/root/data/aquabyte-images/erko/raw/old/', os.path.basename(t)))
    mask = t.replace('labeled_frames', 'labels')
    mask += '.semantic.png'
    shutil.copy(mask, os.path.join('/root/data/aquabyte-images/erko/raw/old/', os.path.basename(mask)))

In [ ]:
t

In [ ]:
# lazy loading
for mask_path in masks:
    
    img_path = mask_path.replace('.semantic.png', '')
    img_path = img_path.replace('.semantic.jpg', '')
    

# PLOTTING

In [ ]:
import cv2
import random
import matplotlib.pyplot as plt

In [ ]:
SEED = 448

In [ ]:
images = glob.glob('/root/data/aquabyte-images/erko/raw/*/*.semantic.jpg') + glob.glob('/root/data/aquabyte-images/erko/raw/*/*.semantic.png')

In [ ]:
random.seed(SEED)
random.shuffle(images)
cutoff = int(len(images)*0.8)
train = images[:cutoff]
val = images[cutoff:]

In [ ]:
batch_size = 8
steps_per_epoch = len(train) // batch_size
steps_per_epoch_val = len(val) // batch_size
input_shape = (512, 512, 3)
kernel = np.ones((5,5),np.uint8)

In [ ]:
def flip_axis(x, axis):
    x = np.asarray(x).swapaxes(axis, 0)
    x = x[::-1, ...]
    x = x.swapaxes(0, axis)
    return x

In [ ]:
def generator(images, steps_per_epoch, BATCH_SIZE, input_shape):
    i = 0

    img_size = input_shape[0]
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 1), dtype=np.uint8)
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            mask_path = images[j]
            img_path = mask_path.replace('.semantic.png', '')
            img_path = img_path.replace('.semantic.jpg', '')
            
            xb = np.array(Image.open(img_path).resize((input_shape[0], input_shape[1])))
            
            
            mask_img = np.array(Image.open(mask_path).resize((input_shape[0], input_shape[1])))
            
            mask0 = np.zeros((input_shape[0], input_shape[1]))
            
            red, green, blue = mask_img[:,:,0], mask_img[:,:,1], mask_img[:,:,2]
            if mask_path.endswith('png'):
                pink_mask = (red == 255) & (green == 105) & (blue == 180)
            elif mask_path.endswith('jpg'):
                pink_mask = (red == 255) & (green == 105) & (blue == 179)
            
            mask0[pink_mask] = 1
            y0 = mask0
            # y0 = cv2.erode(mask0,kernel,iterations = 1)
            
            if np.random.random() > 0.5:
                xb = flip_axis(xb, 1)
                y0 = flip_axis(y0, 1)
                
            x_batch[ind,...] = xb
            y_batch[ind,...,0] = y0
            
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
train_generator = generator(train, steps_per_epoch, batch_size, input_shape)
val_generator = generator(val, steps_per_epoch_val, batch_size, input_shape)

In [ ]:
for _ in range(5):
    X, Y = train_generator.next()
    
    for b in range(batch_size):
        f, ax = plt.subplots(1, 2, figsize=(20, 10))
        # plt.figure(figsize=(10, 10))
        ax[0].imshow(X[b,...])
        ax[1].imshow(X[b,...])
        ax[1].imshow(Y[b,...,0], alpha=0.3)
        plt.show()